In [1]:
import numpy as np
from PIL import Image
import pandas as pd
import os
import tensorflow as tf
import keras
import cv2
import tensorflow as tf
from keras import backend as K


Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
df=pd.read_csv("C:/Users/jaina/Downloads/ESC-50-master/meta/esc50.csv")

In [3]:
processed_dir="C:/Users/jaina/Downloads/ESC-50-master/pro_dir_images/"

In [4]:
def create_dataset(img_folder):
   
    img_data_array=[]
    class_name=[]
   
    for file in os.listdir(img_folder):    
            image_path= os.path.join(img_folder, file)
            image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
            image=cv2.resize(image, (200, 200),interpolation = cv2.INTER_AREA)
            image=np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            #print(img_data_array)
            cat=df[df["filename"].str[:-4]==file[:-4]].target.values[0]
            class_name.append(cat)
    return img_data_array, class_name

In [5]:
img_data, class_name =create_dataset(processed_dir)

In [6]:
X=np.array(img_data, np.float32)
y=np.array(class_name)

In [7]:
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image

#from kt_utils import *


K.set_image_data_format('channels_last')


In [8]:
def Audio2DConvModel(input_shape):
    
    print(input_shape)
    X_input = Input(input_shape)
    print(X_input)
    X = ZeroPadding2D((3, 3))(X_input)
    X = Conv2D(64, (16, 16), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D(pool_size=(2, 2), name = 'maxpool_0')(X)
    X = Dropout(rate=0.5)(X)

    X = ZeroPadding2D((3, 3))(X)
    X = Conv2D(32, (8, 8), strides = (1, 1), name = 'conv1')(X)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2, 2), name = 'maxpool_1')(X)

    X = ZeroPadding2D((3, 3))(X)
    X = Conv2D(16, (4, 4), strides = (1, 1), name = 'conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='maxpool_2')(X)

    X = ZeroPadding2D((3, 3))(X)
    X = Conv2D(16, (2, 2), strides = (1, 1), name = 'conv3')(X)
    X = BatchNormalization(axis = 3, name = 'bn32')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='maxpool_3')(X)
   
    X = Flatten()(X)
    X = Dense(50, activation='softmax', name='fc_softmax')(X)
    model = Model(inputs = X_input, outputs = X, name='Audio2DConvModel')
    return model

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [11]:
from keras import optimizers
basic_conv_model = Audio2DConvModel(X.shape[1:])
basic_conv_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
basic_conv_model.fit(X_train,y_train, epochs=100,batch_size=None,validation_data=(X_test,y_test))
basic_conv_model.summary()

(200, 200, 3)
Tensor("input_1:0", shape=(?, 200, 200, 3), dtype=float32)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 1600 samples, validate on 400 samples
Epoch 1/100
1600/1600 [==============================] - 82s 51ms/sample - loss: 3.8551 - acc: 0.1050 - val_loss: 16.5708 - val_acc: 0.0550
Epoch 2/100
1600/1600 [==============================] - 64s 40ms/sample - loss: 2.8372 - acc: 0.2431 - val_loss: 4.2017 - val_acc: 0.1125
Epoch 3/100
1600/1600 [==============================] - 63s 39ms/sample - loss: 2.4578 - acc: 0.3269 - val_loss: 3.2418 - val_acc: 0.2025
Epoch 4/100
1600/1600 [==============================] - 63s 40ms/sample - loss: 2.0930 - acc: 0.4212 - val_loss: 3.0680 - val_acc: 0.2675
Epoch 5/100
1600/1600 [==============================] - 64s 40ms/sample - loss: 1.6272 - acc: 0.5294 - val_loss: 2.6623 - val_acc: 0.3250
Epoch 6/100
1600/1600 [==============================] - 65s 40ms/sample - loss: 1.1295 - acc: 0.6744 -